In [ ]:
!rm -rf /content/*
%pip install --upgrade --no-cache-dir gdown
!gdown https://drive.google.com/uc?id=1VIKqcHtcBpQlwSKB3h7Q_Z-kDVlr21IC
!gdown https://drive.google.com/uc?id=1732b-_TwlzPJFKc0FoyEW-rfCNCS1heC

In [ ]:
%pip install prophet
%pip install streamlit==1.13.0
%pip install flask-ngrok

In [ ]:
from flask import Flask,request
from flask_ngrok import run_with_ngrok
import pandas as pd
import numpy as np
from prophet import Prophet

app = Flask(__name__)
run_with_ngrok(app)
model = Prophet()
df = pd.read_csv('district-wise-rainfall-data-for-india-2014.csv',sep=';')
df.columns = ['INDIAN_STATES_NAME', 'DISTRICTS_NAME', 'DATE', 'FREQUENCY','VARIABLE_NAME', 'VALUE', 'VARIABLE_NOTES']
df['DATE'] = pd.to_datetime(df['DATE'])
s = '% Dep. are the Departures of rainfall from the long period averages of rainfall for the District.'
df = df[df.VARIABLE_NOTES != s]

def prediction(state,district,d):
    data = df[df['INDIAN_STATES_NAME']==state]
    data = data[data['DISTRICTS_NAME'] == district]
    data = data[['DATE','VALUE']]
    data.index= data['DATE']
    data = data.drop(columns=['DATE'])
    idx = pd.date_range(min(data.index), max(data.index),freq='MS')
    data = data.reindex(idx, fill_value=np.nan)
    data = data.reset_index()
    data = data.rename(columns={'index':'ds', 'VALUE':'y'})
    model = Prophet()
    model.fit(data)
    future_date = pd.DataFrame({'ds':[d]})
    forecast = model.predict(future_date)
    # forecast = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
    return np.round(forecast[['yhat']].values[0][0])

@app.route("/")
def home():
	return {'message':'server is on'}

@app.route("/predict",methods=['POST'])
def predict():
    data = request.json
    print(data)
    state = data['state']
    district = data['district']
    d = data['date']
    pred = prediction(state,district,d)
    return {'prediction':pred}
	
app.run()